# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of skills that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

We encourage you to implement your own versions of the planner that fit your user needs.

Read more about it [here](https://aka.ms/sk/concepts/planner)

In [37]:
!python -m pip install semantic-kernel==0.3.1.dev0

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI backend used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
#if useAzureOpenAI:
#    deployment,api_key,endpoint = sk.azure_openai_settings_from_dot_env()
#    kernel.add_chat_service(
#        "chat-gpt", sk_oai.AzureChatCompletion("gpt35turbo",endpoint,api_key)
#    )
print("Kernel configured")
print("Kernel ready to use")
print("Try typing 'dv' and press TAB")
print(deployment, api_key, endpoint)

Kernel configured
Kernel ready to use
Try typing 'dv' and press TAB
text-davinci-003 9505541cb8204478b05c37c553dcd94d https://llm-langchain.openai.azure.com/


### Setting Up the Planner
Here we will be using a BasicPlanner that will create a sequential plan to be evaluated in order. The output of a function becomes the inputs of the next function. 

In [39]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [40]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

In [41]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""
original_plan = await planner.create_plan_async(ask, kernel)

In [42]:
print(original_plan.generated_plan)

    {
        "input": "Valentine's Day Date Ideas",
        "subtasks": [
            {"function": "WriterSkill.Brainstorm"},
            {"function": "WriterSkill.Translate", "args": {"language": "French"}},
            {"function": "TextSkill.uppercase"}
        ]
    }


You can see that the Planner took my ask and converted it into an JSON-based plan detailing
how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call 
in order to fulfill the user ask. The output of each step of the plan becomes the `input` to the next function. 

Let's also define an inline skill and have it be available to the Planner.
Be sure to give it a function name and skill name.

In [43]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

Let's update our ask using this new skill.

In [44]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel,prompt=ask)

In [26]:
print(new_plan.generated_plan)

DEMAIN EST LA SAINT-VALENTIN. JE DOIS TROUVER QUELQUES IDÉES DE RENDEZ-VOUS. ELLE AIME SHAKESPEARE, DONC ÉCRIVONS DANS SON STYLE. ELLE PARLE FRANÇAIS, DONC ÉCRIVONS EN FRANÇAIS.

FAISONS UNE PROMENADE ROMANTIQUE AU CLAIR DE LUNE DANS UN PARC, OÙ NOUS POURRONS DISCUTER DE NOS SENTIMENTS LES PLUS PROFONDS.

ASSISTONS À UNE PIÈCE DE THÉÂTRE DE SHAKESPEARE, TELLE QUE "ROMÉO ET JULIETTE", SUIVIE D'UN DÎNER AUX CHANDELLES DANS UN RESTAURANT FRANÇAIS INTIME.

VISITONS UN MUSÉE D'ART, ADMIRONS LES OEUVRES CLASSIQUES ET CONTEMPLONS LEUR BEAUTÉ ENSEMBLE, SUIVI D'UN VERRE DE VIN FRANÇAIS DANS UN BAR CHIC.

AIMONS-NOUS LES UNS LES AUTRES COMME SHAKESPEARE L'A ÉCRIT, ET CÉLÉBRONS CETTE JOURNÉE DE L'AMOUR AVEC STYLE ET ROMANTISME.


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [46]:
results = await planner.execute_plan_async(new_plan, kernel)

TypeError: string indices must be integers, not 'str'

In [ ]:
print(results)

# Let's see how the planner would work with a ChatCompletion model

Planner works best with more powerful models like `gpt4`, but because that's a ChatCompletion model, we need to modify the kernel to use that instead. Let's see how it works with the cheaper `gpt-35-turbo` model.

In [11]:
import semantic_kernel as sk
#from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
import semantic_kernel.connectors.ai.open_ai as sk_oai
kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service(
        "chat-gpt", sk_oai.AzureChatCompletion("gpt35turbo",endpoint,api_key)
    )

In [12]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

In [13]:
ask = """
I need to plan a birthday party for my best friend. He loves racecars and only speaks Spanish.
Translate the ideas and make the text in all capital letters."""
chatgpt_plan = await planner.create_plan_async(ask, kernel)

In [14]:
print(chatgpt_plan.generated_plan)

Error: (<ErrorCodes.ServiceError: 6>, 'OpenAI service failed to complete the chat', InvalidRequestError(message='The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.', param=None, code='DeploymentNotFound', http_status=404, request_id=None))


In [15]:
results = await planner.execute_plan_async(chatgpt_plan, kernel)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
print(results)